In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve

In [49]:
train_data = pd.read_csv("Train.csv", index_col = 0)
Test = pd.read_csv("Test.csv", index_col = 0)

In [50]:
train_data["MISSING BUILDING DATA"] = list(map(int, train_data["MISSING BUILDING DATA"]))

In [51]:
for feature in train_data.select_dtypes("object").columns:
    train_data[feature].fillna(train_data[feature].mode()[0], inplace = True)

In [52]:
train_data = pd.get_dummies(train_data)

In [53]:
for feature in train_data.select_dtypes("float64").columns:
    train_data[feature].fillna(train_data[feature].mean(), inplace = True)
    
for feature in train_data.select_dtypes("int64").columns:
    train_data[feature].fillna(train_data[feature].median(), inplace = True)

In [54]:
Test =  pd.get_dummies(Test)
for feature in Test.select_dtypes("float64").columns:
    Test[feature].fillna(train_data[feature].mean(), inplace = True)
    
for feature in Test.select_dtypes("int64").columns:
    Test[feature].fillna(train_data[feature].median(), inplace = True)

In [55]:
train_data.shape

(307511, 228)

In [56]:
Test.shape

(48744, 224)

In [57]:
def fun(y,x):
    for i in x:
        if i not in y:
            print(i)

In [58]:
fun(Test,train_data)

TARGET
CODE_GENDER_XNA
NAME_INCOME_TYPE_Maternity leave
NAME_FAMILY_STATUS_Unknown


In [59]:
train_data = train_data.drop(['CODE_GENDER_XNA', 'NAME_INCOME_TYPE_Maternity leave', 'NAME_FAMILY_STATUS_Unknown'], 1)

In [60]:
train_data.shape

(307511, 225)

In [61]:
train_data

,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,LANDAREA_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,MISSING BUILDING DATA,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,0.036900,0.972200,0.619200,0.014300,0.069000,0.083300,0.036900,0.019000,0.000000,0,...,0,0,0,0,0,0,1,0,1,0
100003,0.052900,0.985100,0.796000,0.060500,0.034500,0.291700,0.013000,0.054900,0.003900,0,...,0,1,0,0,0,0,0,0,1,0
100004,0.088442,0.977735,0.752471,0.044621,0.149725,0.226282,0.066333,0.107399,0.008809,1,...,0,0,0,0,0,1,0,0,1,0
100006,0.088442,0.977735,0.752471,0.044621,0.149725,0.226282,0.066333,0.107399,0.008809,1,...,0,0,0,0,0,1,0,0,1,0
100007,0.088442,0.977735,0.752471,0.044621,0.149725,0.226282,0.066333,0.107399,0.008809,1,...,0,0,0,0,0,1,0,0,1,0
100008,0.088442,0.977735,0.752471,0.044621,0.149725,0.226282,0.066333,0.107399,0.008809,1,...,0,0,0,0,0,1,0,0,1,0
100009,0.088442,0.977735,0.752471,0.044621,0.149725,0.226282,0.066333,0.107399,0.008809,1,...,0,0,0,0,0,1,0,0,1,0
100010,0.088442,0.977735,0.752471,0.044621,0.149725,0.226282,0.066333,0.107399,0.008809,1,...,0,0,0,0,0,1,0,0,1,0
100011,0.088442,0.977735,0.752471,0.044621,0.149725,0.226282,0.066333,0.107399,0.008809,1,...,0,0,0,0,0,1,0,0,1,0


In [62]:
k_fold = KFold(n_splits = 10, random_state = 42, shuffle = True)
folds = []
for train_index, test_index in k_fold.split(train_data):
    to_append = {"Train" : train_data.iloc[train_index,], "Test" : train_data.iloc[test_index]}
    folds.append(to_append)

In [65]:
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

## Logistic Regression

In [66]:
roc_scores = []
for fold in folds:
    log_reg = LogisticRegression(C = 0.0001)
    log_reg.fit(fold["Train"].drop(["TARGET"], axis = 1), fold["Train"]["TARGET"])
    probabilities = log_reg.predict_proba(fold["Test"].drop(["TARGET"], axis = 1))
    roc_scores.append(roc_auc_score(fold["Test"]["TARGET"], probabilities[:, 1]))
    print("Hey")

Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey


In [67]:
sum(roc_scores) / len(roc_scores)

0.6166448087998891

In [68]:
Test_or= pd.read_csv("application_test.csv")

In [69]:
result = Test_or['SK_ID_CURR']
P = log_reg.predict_proba(Test)
preds = pd.DataFrame()
preds['SK_ID_CURR'] = result
l = []
for i in P:
    l.append(i[1])
preds['TARGET'] = l
preds.to_csv('logres.csv', index = False)

## LGBM

In [71]:
roc_scores_lgbm = []
for fold in folds:
    clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1, )
    clf.fit(fold["Train"].drop(["TARGET"], axis = 1), fold["Train"]["TARGET"])
    probabilities = clf.predict_proba(fold["Test"].drop(["TARGET"], axis = 1))
    roc_scores_lgbm.append(roc_auc_score(fold["Test"]["TARGET"], probabilities[:, 1]))
    print("Hey")

LightGBMError: Do not support special JSON characters in feature name.